# Entrenamiento de las LSTM con los datos de 3' UTR <a class="tocSkip"></a>

Cargamos primero las librerías necesarias para la realización del entrenamiento:

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Activation, Conv1D, Flatten, Lambda, Bidirectional, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import keras
import keras.backend as K
import pickle
import os
import json
import pydot
import graphviz

Using TensorFlow backend.


**Importante!!!**

Cambiar el parámetro *allow_growth* es necesario para poder usar la GPU a su total capacidad. También es importante que los imports de arriba del todo sean tensorflow.keras y no keras solo (si no no va).

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1.keras.backend import set_session

config = ConfigProto()
config.gpu_options.allow_growth = True # dynamically grow the memory used on the GPU
session = InteractiveSession(config=config) # to log device placement (on which device the operation ran)
                                            # (nothing gets printed in Jupyter, only if you run it standalone)
set_session(session) # set this TensorFlow session as the default session for Keras

## Entrenamiento usando las cadenas reversed

### Partición del conjunto

In [3]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
readRDS = robjects.r['readRDS']


/home/rafajorda/.conda/envs/tfm_env/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Leemos los objetos de train y test obtenidos del .Rmd de Jaime:

In [4]:
trainx = readRDS('../trainx.rds')
testx = readRDS('../testx.rds')
trainy = readRDS('../trainy.rds')
testy = readRDS('../testy.rds')

In [15]:
np.array(trainx[2]).shape

(24289, 808, 1)

### Entrenamiento de modelos

Definimos la función que va a construir el modelo LSTM. Este modelo puede tener una o dos capas LSTM, en función de los parámetros de entrada que se le introducen. Si tiene una capa LSTM, tendrá además una capa de dropout, y si tiene dos capas LSTM, tendrá dos capas de dropout. La última capa del modelo es `softmax` con 2 nodos, luego nuestra salida deberá estar codificada en one-hot. El otro parámetro del modelo es el learning rate. Además, se usa como medida de loss la cross-entropía binaria, y como métrica el accuracy.

In [29]:
def create_lstm(nodes1, dropout, lr, two_layers = False, nodes2 = 32):
    model = Sequential()
    if two_layers:
        model.add(Bidirectional(LSTM(nodes1, return_sequences = True, input_shape = (202,4))))
    else:
        model.add(Bidirectional(LSTM(nodes1, input_shape = (202,4))))
    model.add(Dropout(dropout))
    if two_layers:
        model.add(Bidirectional(LSTM(nodes2)))
        model.add(Dropout(dropout))
    model.add(Dense(2, activation = "softmax"))
    opt = Adam(learning_rate=lr)
    model.compile(optimizer = opt, loss = "binary_crossentropy", 
                  metrics = ["accuracy"])
    return model

Guardamos las variables de entrada y salida de la primera partición de las que hizo Jaime:

In [20]:
trrx = np.array(trainx[0])
tex = np.array(testx[0])
trry = np.array(trainy[0])
tey = np.array(testy[0])

Cambiamos las secuencias para que los vectores one-hot sean de longitud 4:

In [21]:
trrx = np.array([np.array(seq).reshape(202,4) for seq in trrx])
tex = np.array([np.array(seq).reshape(202,4) for seq in tex])

Reordenamos aleatoriamente los conjuntos de entrenamiento y test (no tiene por qué hacerse):

In [22]:
ty = pd.Series(np.squeeze(trry)).sample(len(np.squeeze(trry)), random_state = 1)
tx = trrx[ty.index]
ty = ty.values

In [23]:
vy = pd.Series(np.squeeze(tey)).sample(len(np.squeeze(tey)), random_state = 1)
vx = tex[vy.index]
vy = vy.values

Realizamos el entrenamiento con 1 capa, 128 nodos y un learning rate de 0.001:

In [24]:
batch_size = 125
epochs = 25
dropout = 0
nodos1 = 128
lr = 0.001
# Cargamos el modelo y lo compilamos
model = create_lstm(nodos1, dropout, lr)
# Ejecutamos el modelo
# Con la función to_categorical convertimos el vector de salida a one-hot
historyIt = model.fit(tx,to_categorical(ty), batch_size= batch_size, epochs=epochs,
                    validation_data = (vx, to_categorical(vy)))

Train on 24289 samples, validate on 6073 samples
Epoch 1/25
24289/24289 [==============================] - 8s 316us/sample - loss: 0.4199 - accuracy: 0.8052 - val_loss: 0.2632 - val_accuracy: 0.8918
Epoch 2/25
24289/24289 [==============================] - 5s 217us/sample - loss: 0.2369 - accuracy: 0.9085 - val_loss: 0.1805 - val_accuracy: 0.9335
Epoch 3/25
24289/24289 [==============================] - 5s 219us/sample - loss: 0.1870 - accuracy: 0.9294 - val_loss: 0.1617 - val_accuracy: 0.9391
Epoch 4/25
24289/24289 [==============================] - 5s 217us/sample - loss: 0.1762 - accuracy: 0.9334 - val_loss: 0.1485 - val_accuracy: 0.9447
Epoch 5/25
24289/24289 [==============================] - 5s 217us/sample - loss: 0.1632 - accuracy: 0.9411 - val_loss: 0.1485 - val_accuracy: 0.9458
Epoch 6/25
24289/24289 [==============================] - 5s 215us/sample - loss: 0.1662 - accuracy: 0.9375 - val_loss: 0.1424 - val_accuracy: 0.9485
Epoch 7/25
24289/24289 [===========================

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection multiple                  136192    
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  514       
Total params: 136,706
Trainable params: 136,706
Non-trainable params: 0
_________________________________________________________________
